## Quartic Potential
For convenience the key equations are given below from https://arxiv.org/pdf/astro-ph/9407016.pdf:

1) Fokker-Planck equation
$$    \frac{\partial P(\varphi,t)}{\partial t} = \frac{1}{3H}\frac{\partial}{\partial \varphi}\left(V'(\varphi) P(\varphi,t)\right) + \frac{H^3}{8\pi^2}\frac{\partial^2 P(\varphi,t)}{\partial \varphi^2} $$

2) Schrodinger-like equation
$$     -\frac{1}{2}\frac{\partial^2 \Phi_n(\varphi)}{\partial \varphi^2} + W(\varphi) \Phi_n (\varphi) = \frac{4\pi^2\Lambda_n}{H^3}\Phi_n(\varphi) $$

3) Effective potential
$$ W(\varphi) = \frac{1}{2}\left[v'(\varphi)^2-v''(\varphi)\right], \> \> v(\varphi) = \frac{4\pi^2}{3H^4} V(\varphi) $$

4) Solutions take the form
$$ P(\varphi,t) = \exp\left(-\frac{4\pi^2 V(\varphi)}{3H^4}\right) \sum^{\infty}_{n=0} a_n \Phi_n (\varphi) e^{-\Lambda_n (t-t_0)} $$

In [1]:
import numpy as np
from math import factorial, ceil
from scipy import optimize
from scipy.integrate import odeint, simps
from scipy.special import eval_hermite
import matplotlib.pyplot as plt
%matplotlib inline

## Defining key parameters 

In [2]:
hubble_rate = 1
mass = 0
mass_squared = mass**2
self_coupling = 1

For a self-interacting field, the interactions may be characterised by the following potentials
$$V(\varphi) = \frac{1}{2}m^2\varphi^2 + \frac{1}{4}\lambda\varphi^4 $$
$$    W(\varphi) = m^2\frac{2\pi^2}{3H^4}\left[\frac{4\pi^2}{3H^4} m^2 \varphi^2 - 1\right] + \lambda\frac{2\pi^2}{3H^4}\left[\frac{4\pi^2}{3H^4} \lambda \varphi^6 - 3 \varphi^2 \right] $$

## Initial eigenfunction entries

In [3]:
# Even eigenfunction starting conditions
V_even = [1, 0]
# Odd eigenfunction starting conditions
V_odd = [0, 1]

In [4]:
class eigenfunction_finder:
    '''Class for finding the eigenfunction for a given eigenvalue'''
    def __init__(self, V, field):
        '''Initialises the range of field values for which the field is
        to be integrated
        V: array where first entry is the eigenfunction, second is Y
        as defined for the coupled ODE system
        field: field values'''
        self.V = V
        self.field = field        
 
    def effective_potential(self, field):
        '''Defines the effective potential in the SL equation'''
    
        # Contribution to effective potential of quartic and squared terms respectively
        quartic_potential = self_coupling*((2*np.pi**2)/(3*hubble_rate**4))*(((4*np.pi**2)/(3*hubble_rate**4))*self_coupling*field**6 - 3*field**2)
        quadratic_potential = mass_squared*((2*np.pi**2)/(3*hubble_rate**4))*(((4*np.pi**2)/(3*hubble_rate**4))*mass_squared*field**2 - 1)
        
        W = quadratic_potential + quartic_potential
        return W


    def schrodinger_like_ODE(self, V, field):
        '''Computes the derivatives of the Schrodinger-like equation'''

        # Empty array for derivatives
        dV = np.zeros(2)
        # Computes derivatives
        dV[0] = V[1]
        dV[1] = V[0]*(2*self.effective_potential(field) - (8*self.eigenvalue*np.pi**2)/(hubble_rate**3))
        return dV    

    def eigenfunction(self, eigenvalue):
        '''Integrates the eigenfunction for the field based on the given
        field range and returns the projected solution based on the
        eigenvalue'''
        
        self.eigenvalue = eigenvalue
        
        # Integrates to find eigenfunction
        V = odeint(self.schrodinger_like_ODE, self.V, self.field)
        
        return V[-1,0]
    
    def eigenfunction_full(self, eigenvalue):
        '''Integrates the eigenfunction for the field based on the given
        field range and returns the projected solution based on the
        eigenvalue'''
        
        self.eigenvalue = eigenvalue
        
        # Integrates to find eigenfunction
        V = odeint(self.schrodinger_like_ODE, self.V, self.field)

        return V[:,0]

In [5]:
# Field 
field = np.linspace(0, 3.5, 3000)

# Tolerance 
tolerance = 1e-6

# Scipy implementations of the Brent for n = 0
eigenvalue_function = eigenfunction_finder(V_even, field)
scipy_brent_0, results_brent_0 = optimize.brentq(eigenvalue_function.eigenfunction,-0.1, 0.2, full_output=True, xtol=tolerance)
scipy_brent_V_0 = eigenvalue_function.eigenfunction_full(scipy_brent_0)
print(results_brent_0)

      converged: True
           flag: 'converged'
 function_calls: 16
     iterations: 15
           root: 8.04061562588124e-09


In [6]:
# Identifying a suitable guess interval for n = 1
eigenvalue_function = eigenfunction_finder(V_odd, field)
print(eigenvalue_function.eigenfunction_full(0.1)[-1])
print(eigenvalue_function.eigenfunction_full(0.3)[-1])
print(eigenvalue_function.eigenfunction_full(0.5)[-1])
print(eigenvalue_function.eigenfunction_full(0.7)[-1])

-1.424442390981696e+210
-5.445426016607305e+210
-5.572774538066781e+209
1.0936220813325822e+210


In [7]:
# Scipy implementations of the Brent algoritms for n = 1
eigenvalue_function = eigenfunction_finder(V_odd, field)
scipy_brent_1, results_brent_1 = optimize.brentq(eigenvalue_function.eigenfunction, 0.5, 0.7, full_output=True, xtol=tolerance)
scipy_brent_V_1 = eigenvalue_function.eigenfunction_full(scipy_brent_1)
print(results_brent_1)

      converged: True
           flag: 'converged'
 function_calls: 12
     iterations: 11
           root: 0.5366714470633597


In [8]:
# Identifying a suitable guess interval for n = 2
eigenvalue_function = eigenfunction_finder(V_even, field)
print(eigenvalue_function.eigenfunction_full(0.6)[-1])
print(eigenvalue_function.eigenfunction_full(0.9)[-1])

1.2102284580184852e+211
-2.287430507436089e+210


In [9]:
# Scipy implementations of the Brent algoritms for n = 2
eigenvalue_function = eigenfunction_finder(V_even, field)
scipy_brent_2, results_brent_2 = optimize.brentq(eigenvalue_function.eigenfunction, 0.6, 0.9, full_output=True, xtol=tolerance)
scipy_brent_V_2 = eigenvalue_function.eigenfunction_full(scipy_brent_2)
print(results_brent_2)

      converged: True
           flag: 'converged'
 function_calls: 15
     iterations: 14
           root: 0.8289521829570561


Note that as I my guess intervals skipped some of the eigenvalues the $n = 1$, and $n =2$ cases above are actually those for $n=3$ and $n=4$ respectively. The actual $n=1,2$ cases are given below.

In [10]:
# Scipy implementations of the Brent algoritms for n = 1
eigenvalue_function = eigenfunction_finder(V_odd, field)
scipy_brent_1, results_brent_1 = optimize.brentq(eigenvalue_function.eigenfunction, 0, 0.1, full_output=True, xtol=tolerance)
scipy_brent_V_1 = eigenvalue_function.eigenfunction_full(scipy_brent_1)
print(results_brent_1)

      converged: True
           flag: 'converged'
 function_calls: 13
     iterations: 12
           root: 0.08892383530399899


In [11]:
# Scipy implementations of the Brent algoritms for n = 2
eigenvalue_function = eigenfunction_finder(V_even, field)
scipy_brent_2, results_brent_2 = optimize.brentq(eigenvalue_function.eigenfunction, 0.2, 0.4, full_output=True, xtol=tolerance)
scipy_brent_V_2 = eigenvalue_function.eigenfunction_full(scipy_brent_2)
print(results_brent_2)

      converged: True
           flag: 'converged'
 function_calls: 13
     iterations: 12
           root: 0.28937856555307656


In [12]:
class brent:
    '''Root finding class with the aforementioned
    algorithms implemented as methods'''
    def __init__(self, interval):
        '''Initialization of eigenvalues where the bracketed eigenvalues
        are assigned as attributes.
        interval: two element array where the first entry is the lower 
        eigenvalue and the second element is the higher eigenvalue'''
        
        # Setting up initial trial eigenvalues
        self.initial_low_eigenvalue = interval[0]
        self.initial_high_eigenvalue = interval[1]
    
    def effective_potential(self, field):
        '''Defines the effective potential in the SL equation
        field: field value'''
        
        # Contribution to effective potential of quartic and squared terms respectively
        quartic_potential = self_coupling*((2*np.pi**2)/(3*hubble_rate**4))*(((4*np.pi**2)/(3*hubble_rate**4))*self_coupling*field**6 - 3*field**2)
        quadratic_potential = mass_squared*((2*np.pi**2)/(3*hubble_rate**4))*(((4*np.pi**2)/(3*hubble_rate**4))*mass_squared*field**2 - 1)
        
        W = quadratic_potential + quartic_potential
        return W
    
    def schrodinger_like_ODE(self, V, field):
        '''Computes the derivatives of the Schrodinger-like equation
        V: array where first entry is the eigenfunction, second is Y
        as defined for the coupled ODE system
        field: field values'''
    
        # Empty array for derivatives
        dV = np.zeros(2)

        # Computes derivatives
        dV[0] = V[1]
        dV[1] = V[0]*(2*self.effective_potential(field) - (8*self.eigenvalue*np.pi**2)/(hubble_rate**3))
        return dV    

    def brent_loop(self, V, field, tolerance, max_iterations):
        '''Implements Brents method to compute eigenvalues.
        V: array where first entry is the eigenfunction, second is Y
        as defined for the coupled ODE system
        field: field values
        tolerance: determines how close to zero the solutions should reach 
        to be deemed to have converged sufficiently
        max_iterations: caps the number of iterations the loop undergoes
        in the event the tolerance is not reached'''
        
         # Initial low eigenvalue and its solution
        self.high_eigenvalue = self.initial_high_eigenvalue
        self.eigenvalue = self.high_eigenvalue
        V_high = odeint(self.schrodinger_like_ODE, V, field)
        self.function_calls_brent = 1
    
        # Initial high eigenvalue and its solution
        self.low_eigenvalue = self.initial_low_eigenvalue
        self.eigenvalue = self.low_eigenvalue
        V_low = odeint(self.schrodinger_like_ODE, V, field)
        self.function_calls_brent += 1
        
        # Initial midpoint eigenvalue and its solution
        self.new_eigenvalue = 0.5*(self.high_eigenvalue + self.low_eigenvalue)
        self.eigenvalue = self.new_eigenvalue
        V_new = odeint(self.schrodinger_like_ODE, V, field)
        self.function_calls_brent += 1
        
        # Initializing number of iterations
        iterations = 0
        # Loop that runs until we hit the desired tolerance or max_iterations is exceeded
        while self.high_eigenvalue - self.low_eigenvalue > tolerance:
            # Computes parameters associated with Brents' method
            R = V_new[-1,0]/V_high[-1,0]
            S = V_new[-1,0]/V_low[-1,0]
            T = V_low[-1,0]/V_high[-1,0]
            P = S*(T*(R - T)*(self.high_eigenvalue - self.new_eigenvalue) - (1 - R)*(self.new_eigenvalue - self.low_eigenvalue))
            Q = (T - 1)*(R - 1)*(S - 1)
            
            # Next guess for root based on Brent's method
            next_eigenvalue = self.new_eigenvalue + P/Q
            # Compares convergence with bisection
            if abs(next_eigenvalue - self.new_eigenvalue) < (self.initial_high_eigenvalue - self.initial_low_eigenvalue)*0.5**(iterations+2):
                # Checks to see which half of the current bounds the next guess lies then redefines the bounds
                if next_eigenvalue < self.new_eigenvalue:
                    self.high_eigenvalue = self.new_eigenvalue
                    self.eigenvalue = self.high_eigenvalue
                    V_high = V_new #odeint(self.schrodinger_like_ODE, V, field)
                    #self.function_calls_brent += 1
    
                else:
                    self.low_eigenvalue = self.new_eigenvalue
                    self.eigenvalue = self.low_eigenvalue
                    V_low = V_new # odeint(self.schrodinger_like_ODE, V, field)
                    #self.function_calls_brent += 1
                
                # Updates guess of root
                self.new_eigenvalue = next_eigenvalue
                self.eigenvalue = self.new_eigenvalue
                V_new = odeint(self.schrodinger_like_ODE, V, field)
                self.function_calls_brent += 1
            
            # Tries bisection if convergence was deemed too slow
            else:
                # Checks to see which boundary eigenvalue gives positive or negative diverging solutions
                if V_high[-1,0] > V_new[-1,0] > V_low[-1,0]:
                    # Checks if the midpoint eigenvalue gives negatively diverging solutions
                    if V_new[-1,0] < 0:                    
                        # Sets eigenvalue c to be eigenvalue a, likewise for the solutions
                        V_low = V_new
                        self.low_eigenvalue = self.new_eigenvalue
                    
                        # Determines the new midpoint eigenvalue and its solutions
                        self.new_eigenvalue = (self.high_eigenvalue + self.low_eigenvalue)/2
                        self.eigenvalue = self.new_eigenvalue
                        V_new = odeint(self.schrodinger_like_ODE, V, field)
                        self.function_calls_brent += 1
                        
                    # Runs if the midpoint eigenvalue converges to positive infinity
                    else:
                        # Sets eigenvalue c to be eigenvalue b, likewise for the solutions
                        V_high = V_new
                        self.high_eigenvalue = self.new_eigenvalue
                    
                        # Determines the new midpoint eigenvalue and its solutions
                        self.new_eigenvalue = (self.high_eigenvalue + self.low_eigenvalue)/2
                        self.eigenvalue = self.new_eigenvalue
                        V_new = odeint(self.schrodinger_like_ODE, V, field)
                        self.function_calls_brent += 1
                        
                # Runs if the higher eigenvalue gives negative diverging solutions while the lower gives the reverse
                else:
                    # Checks if the midpoint eigenvalue gives negatively diverging solutions
                    if V_new[-1,0] < 0:
                        # Sets eigenvalue c to be eigenvalue b, likewise for the solutions
                        V_high = V_new
                        self.high_eigenvalue = self.new_eigenvalue
                    
                        # Determines the new midpoint eigenvalue and its solutions
                        self.new_eigenvalue = (self.high_eigenvalue + self.low_eigenvalue)/2
                        self.eigenvalue = self.new_eigenvalue
                        V_new = odeint(self.schrodinger_like_ODE, V, field)
                        self.function_calls_brent += 1
                        
                    # Runs if the midpoint eigenvalue converges to positive infinity
                    else:
                        # Sets eigenvalue c to be eigenvalue a, likewise for the solutions
                        V_low = V_new
                        self.low_eigenvalue = self.new_eigenvalue
                    
                        # Determines the new midpoint eigenvalue and its solutions
                        self.new_eigenvalue = (self.high_eigenvalue + self.low_eigenvalue)/2
                        self.eigenvalue = self.new_eigenvalue
                        V_new = odeint(self.schrodinger_like_ODE, V, field)
                        self.function_calls_brent += 1
            
            # Updates number of iterations
            iterations = iterations + 1
            # If max_iterations is exceeded the loop is stopped
            if iterations >= max_iterations:
                break
                
        return self.new_eigenvalue, V_new, iterations

In [13]:
# Field 
field = np.linspace(0, 3.5, 3000)

# Tolerance
tolerance = 1e-6

# Running the Brent algorithms for n = 0
brent_solver = brent([-0.1, 0.2])
eigenvalue_brent_0, V_brent_0, iterations_brent_0 = brent_solver.brent_loop(V_even, field, tolerance, 100)
brent_function_calls_0 = brent_solver.function_calls_brent

# Running the secant/bisection/Ridder/Brent algorithm for n = 1
brent_solver = brent([0, 0.1])
eigenvalue_brent_1, V_brent_1, iterations_brent_1 = brent_solver.brent_loop(V_odd, field, tolerance, 100)
brent_function_calls_1 = brent_solver.function_calls_brent

# Running the secant/bisection/Ridder/Brent algorithm for n = 2
brent_solver = brent([0.2, 0.4])
eigenvalue_brent_2, V_brent_2, iterations_brent_2 = brent_solver.brent_loop(V_even, field, tolerance, 100)
brent_function_calls_2 = brent_solver.function_calls_brent

In [14]:
print(eigenvalue_brent_0, eigenvalue_brent_1, eigenvalue_brent_2)
print(iterations_brent_0, iterations_brent_1, iterations_brent_2)
print(brent_function_calls_0, brent_function_calls_1, brent_function_calls_2)

1.4869033898820694e-10 0.08892393083553396 0.28937854849714445
7 6 6
10 9 9
